In [3]:
pip install datasets


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from datasets import load_dataset



In [2]:
from huggingface_hub import login
login()

In [37]:
# 1. تحميل بيانات AG News
import requests
import pandas as pd

# اخذ الداتا من API
url = "https://datasets-server.huggingface.co/rows?dataset=wangrongsheng%2Fag_news&config=default&split=train&offset=0&length=100"

response = requests.get(url)

data = response.json()  # convert res to JSON
rows = data["rows"]  # استخراج الداتا الفعلية
df = pd.DataFrame([row["row"] for row in rows])
df

,text,label
0,Wall St. Bears Claw Back Into the Black (Reute...,2
1,Carlyle Looks Toward Commercial Aerospace (Reu...,2
2,Oil and Economy Cloud Stocks' Outlook (Reuters...,2
3,Iraq Halts Oil Exports from Main Southern Pipe...,2
4,"Oil prices soar to all-time record, posing new...",2
...,...,...
95,Gene Blocker Turns Monkeys Into Workaholics - ...,3
96,Dolphins Too Have Born Socialites (Reuters) Re...,3
97,"What's in a Name? Well, Matt Is Sexier Than Pa...",3
98,UK Scientists Allowed to Clone Human Embryos (...,3


In [49]:
df_label_3 = df[df['label'] == 3]
df_label_3.count()

text     22
label    22
dtype: int64

In [38]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer  # تحويل النصوص إلى ميزات عددية
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score, classification_report

In [39]:
def preprocess_text(text):
    """تنظيف النصوص عبر تحويلها إلى حروف صغيرة فقط."""
    return text.lower()

In [40]:
# تطبيق المعالجة المسبقة على النصوص
df["text"] = df["text"].apply(preprocess_text)

In [41]:
# 2. تقسيم البيانات إلى train and test
X_train, X_test, y_train, y_test = train_test_split(
    df["text"], df["label"], test_size=0.2, random_state=42
)

In [43]:
# 3. convert text into Numerical features usnig TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)  # select 5000 feature (لزيادة الكفاءة)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [44]:
# 4. تدريب عدة نماذج ومقارنتها
models = {
    "Logistic Regression": LogisticRegression(),
    "Decision Tree": DecisionTreeClassifier(),
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "AdaBoost": AdaBoostClassifier()
}

In [45]:
# تخزين نتائج الأداء
results = {}

for name, model in models.items():
    print(f"Training {name}...")
    model.fit(X_train_tfidf, y_train)  # تدريب النموذج
    y_pred = model.predict(X_test_tfidf)  # إجراء التنبؤات
    accuracy = accuracy_score(y_test, y_pred)  # حساب الدقة
    results[name] = accuracy
    print(f"{name} Accuracy: {accuracy:.4f}\n")
    print(classification_report(y_test, y_pred))

Training Logistic Regression...
Logistic Regression Accuracy: 0.8500

              precision    recall  f1-score   support

           2       0.85      1.00      0.92        17
           3       0.00      0.00      0.00         3

    accuracy                           0.85        20
   macro avg       0.42      0.50      0.46        20
weighted avg       0.72      0.85      0.78        20

Training Decision Tree...
Decision Tree Accuracy: 0.9500

              precision    recall  f1-score   support

           2       1.00      0.94      0.97        17
           3       0.75      1.00      0.86         3

    accuracy                           0.95        20
   macro avg       0.88      0.97      0.91        20
weighted avg       0.96      0.95      0.95        20

Training KNN...
KNN Accuracy: 0.9500

              precision    recall  f1-score   support

           2       0.94      1.00      0.97        17
           3       1.00      0.67      0.80         3

    accuracy    

C:\WPy64-31220\python-3.12.2.amd64\Lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\WPy64-31220\python-3.12.2.amd64\Lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\WPy64-31220\python-3.12.2.amd64\Lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


AdaBoost Accuracy: 0.9000

              precision    recall  f1-score   support

           2       0.94      0.94      0.94        17
           3       0.67      0.67      0.67         3

    accuracy                           0.90        20
   macro avg       0.80      0.80      0.80        20
weighted avg       0.90      0.90      0.90        20



C:\WPy64-31220\python-3.12.2.amd64\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
